In [ ]:
# Installing the libraries for project
!pip install -U tensorflow==2.9.2
!pip install fastapi==0.90.0
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install spacy-transformers
!python -m spacy download en_core_web_trf


**Before training the anntaions.json needs to be converted docbin format of spacy which is done below. Afterwards training is done**


In [2]:
import os 
import json
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import spacy 
from spacy.tokens import DocBin
from tqdm import tqdm

In [3]:
os.chdir('/content/drive/MyDrive/assigment_bento_labs')


In [4]:
# Code taken from https://github.com/explosion/spaCy/discussions/8456
def make_v3_dataset(data, db = []):
    nlp = spacy.blank('en')
    failed_record = []
    if not db:
        db = DocBin()
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot['entities']:
            span = doc.char_span(start, end, label = label, alignment_mode = 'contract')
            if span is None:
                print(f'empty entity, {text}, {annot["entities"]}') #I expect this to never happen
            else:
                ents.append(span)
        try:
            doc.ents = ents
        except:
            failed_record.append((text, annot))
        db.add(doc)
    return db, failed_record

In [5]:
# Converting json annotated files to spacy format
with open('annotations.json','r') as f:
  data = json.load(f)
  saving_path = os.path.join('.','annotaed_data_spacy_format'+'.spacy')
  a,b = make_v3_dataset(data['annotations'])
  a.to_disk(saving_path)

100%|██████████| 121/121 [00:00<00:00, 4261.00it/s]


In [6]:
# setting up the environment for running cli commands
import locale
print(locale.getpreferredencoding())

ANSI_X3.4-1968


In [7]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [8]:
# Checking avalable  gpu 
!nvidia-smi


Tue Mar 14 11:01:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    29W /  70W |    343MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
# Initialization setting
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [10]:
# Training on labelled data
!python -m spacy train -g 0 config.cfg  --output generated_model/ --paths.train ./annotaed_data_spacy_format.spacy --paths.dev ./annotaed_data_spacy_format.spacy

ℹ Saving to output directory: generated_model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-03-14 11:03:03,035] [INFO] Set up nlp object from config
[2023-03-14 11:03:03,052] [INFO] Pipeline: ['transformer', 'ner']
[2023-03-14 11:03:03,057] [INFO] Created vocabulary
[2023-03-14 11:03:03,058] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initia

In [11]:
# Predicting values on remaining dataset
nlp = spacy.load('generated_model/model-best')
df = pd.read_csv('data/preprocessed_text.csv')

In [30]:
def return_actions(txt):
  actions_list = []
  for doc in nlp.pipe([txt], disable=["tagger", "parser"]):
    for ent in doc.ents:
      if ent.label_=="ACTIONS":
        actions_list.append(ent.text)
  return actions_list

In [31]:
df['actions'] = df.text.apply(return_actions)

In [37]:
df.head(10)

,text,actions
0,she said she not yet decided anything she is n...,"[said, decided, responding, calls, msgs]"
1,he is not responding to calls and msgs,"[responding, calls, msgs]"
2,lead assigned to ramakrishna katta by anshul l...,"[lead, lead]"
3,still number shows off,[off]
4,lead assigned to raj hegde by anshul,[]
5,he said he will call me once he get free he sa...,"[said, call, free, interested, call]"
6,tried calling him again he still not picking u...,"[tried, calling, picking, call, disconnecting,..."
7,lead assigned to ramakrishna katta by anshul l...,"[lead, lead]"
8,tried again still she is not responding to cal...,"[responding, calls, msgs]"
9,tried again still not responding to call rnr t...,"[tried, responding, call, responding, call, re..."


In [38]:
df.to_csv('data/data_with_actions.csv')